In [1]:
pip install sounddevice


Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import sounddevice as sd

fs = 44100
duration = 1.0
freq = 440  # תדר לדוגמה (A)

t = np.linspace(0, duration, int(fs * duration), endpoint=False)
wave = 0.5 * np.sin(2 * np.pi * freq * t)

sd.play(wave, fs)
sd.wait()


In [ ]:
# Small Neural Network with Overfitting + Bayesian Regression Comparison

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

# ----- DATA SETUP -----
# Training data (same as Bayesian model)
X = np.array([1.0, 2.0, 3.0])
Y = np.array([300.0, 400.0, 500.0])

# Test inputs for prediction
x_test = np.linspace(0.0, 4.0, 100).reshape(-1, 1)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X.reshape(-1, 1), dtype=torch.float32)
Y_tensor = torch.tensor(Y.reshape(-1, 1), dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)

# ----- NEURAL NETWORK -----
model = nn.Sequential(
    nn.Linear(1, 50),
    nn.Tanh(),
    nn.Linear(50, 50),
    nn.Tanh(),
    nn.Linear(50, 1)
)

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train neural network
num_epochs = 1000
losses = []
for epoch in range(num_epochs):
    model.train()
    y_pred = model(X_tensor)
    loss = loss_fn(y_pred, Y_tensor)
    losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Predict with neural network
model.eval()
y_test_pred_nn = model(x_test_tensor).detach().numpy()

# ----- BAYESIAN REGRESSION -----
# Feature function: [1, x] for each input
def phi(x):
    return np.array([1.0, x])

# Build design matrix Phi
Phi = np.vstack([phi(x) for x in X]).T  # shape: (2, N)

# Prior parameters
mu_prior = np.zeros(2)
Sigma_prior = np.eye(2)
sigma2 = 25.0  # observation noise variance

# Posterior computation
precision_post = np.linalg.inv(Sigma_prior) + (1 / sigma2) * (Phi @ Phi.T)
Sigma_post = np.linalg.inv(precision_post)
mu_post = Sigma_post @ ((1 / sigma2) * Phi @ Y)

# Predictive mean for each test point
phi_test = np.vstack([phi(x[0]) for x in x_test])  # shape: (N_test, 2)
mu_fx = phi_test @ mu_post

# ----- PLOT -----
plt.figure(figsize=(10, 5))
plt.plot(x_test, y_test_pred_nn, label="Neural Network Prediction", color='r')
plt.plot(x_test, mu_fx, label="Bayesian Prediction", color='g')
plt.scatter(X, Y, color='blue', label="Training Data", zorder=5)
plt.title("Neural Network (Overfitting) vs Bayesian Regression")
plt.xlabel("Input X")
plt.ylabel("Output Y (Frequency)")
plt.legend()
plt.grid(True)
plt.show()
